In [1]:
import sys
import os
current = os.path.dirname(os.path.realpath("Single-House-Optimization.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,10)

import numpy as np
import pandas as pd
from copy import deepcopy
from Merge import merge
from copy import deepcopy
from Logic import logic_rollout, action_rollout, pred_logic_rollout, print_price_summary, logic_series_print
from Battery import Battery
from DPModel import DPModel, DP, DP_stochastic

from P2P_Dynamics import EnergyMarket

In [2]:
Start = '2022-06-19 00:00:00'
End = '2022-06-19 23:00:00'
N = len(pd.date_range(Start, End,freq='H'))

dfA = merge('h16')
dfB = merge('h28')
dfC = merge('k28')

In [3]:
battery = Battery(max_capacity=13, max_charge=7)
series_DP_A = DP(Start, End, dfA, battery, byday=True, ints=True, degrade=False, verbose=True)
series_DP_B = DP(Start, End, dfB, Battery(max_capacity=13, max_charge=7,current_capacity=10), byday=True, ints=True, degrade=False, verbose=True)
series_DP_C = DP(Start, End, dfC, battery, byday=True, ints=True, degrade=False, verbose=True)

Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
0  Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
0  Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
0  

In [4]:
print_price_summary(series_DP_A, False)

The period is from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Cost for period:  150.0  DKK
Total emissions for period:  13.0  kg

Number of kwh purchased in the period: 72.3
Number of kwh sold in the period: 6.5


In [5]:
print_price_summary(series_DP_B, False)

The period is from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Cost for period:  70.0  DKK
Total emissions for period:  8.0  kg

Number of kwh purchased in the period: 38.7
Number of kwh sold in the period: 5.999999999999999


In [6]:
print_price_summary(series_DP_C, False)

The period is from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Cost for period:  173.0  DKK
Total emissions for period:  14.0  kg

Number of kwh purchased in the period: 80.10000000000001
Number of kwh sold in the period: 0.0


In [7]:
class DP_central(DPModel):
    def __init__(self, Start, End, houses, battery,degrade=False,ints=False,acts=None,acts_range=None,
                 furthers=None,traj=None, traj_range=None,max_number_states=200): 
        super().__init__(Start, End, merge(houses[0]), battery,degrade,ints,acts,acts_range)
        self.houses = houses
        
        self.furthers = furthers
        
        self.traj = traj
        self.traj_range = traj_range
        
        self.max_number_states = max_number_states
        
        temp = pd.DataFrame(columns=houses)
        for i in range(len(houses)):
            temp[houses[i]] = merge(houses[i]).loc[Start:End]["yield"]

        self.yields = temp
    
        #Compute state space once
        if ints:
            states = np.arange(0, battery.max_capacity+1, 1)
        else:
            states = np.round(np.arange(0.0, battery.max_capacity+0.01, 0.1),2)
        
        self.states = [states for _ in range(len(houses))]
        
        if self.ints and self.furthers is not None:
            for i in range(len(houses)):
                temp = self.states[i]
                self.states[i] = temp[temp%self.furthers[i][0]==self.furthers[i][1]]

        self.s = np.array(np.meshgrid(*self.states)).T.reshape(-1,3)
        self.s = self.s.tolist()
        self.s = [tuple(x) for x in self.s]
    
    def f(self, x, u, w, k):
        ogbat = self.battery
        self.battery = deepcopy(ogbat)
        
        res = []
        for i in range(len(self.houses)):
            self.battery.current_capacity = x[i]
            self.battery.charge(u[i], degrade=self.degrade)

            if self.ints:
                self.battery.current_capacity = int(self.battery.current_capacity)
            
            res.append(self.battery.get_current_capacity())
            
        self.battery = ogbat
        
        return tuple(res)
    
    def g(self, x, u, w, k):
        yields = self.get_yield(k)
        surpluses = [yields[i]-u[i] for i in range(len(u))]
        
        fee = 1 #transmission fee
        
        participants = {self.houses[i]: surpluses[i] for i in range(len(u))}
    
        em = EnergyMarket(participants, self.sp[k], self.sp[k]+fee)
        
        dic = em.get_total_costs()
        
        return sum([dic[house] for house in self.houses])
    
    def S(self, k):
        if (self.traj is not None) and (self.traj_range is not None):
            for i in range(len(self.houses)):
                if self.traj_range[i]==0.0:
                    self.states[i] = self.traj[k][i]
                
            temp = np.array(np.meshgrid(*self.states)).T.reshape(-1,3)
            temp = temp.tolist()
            temp = [tuple(x) for x in temp]
            
            return temp
        
        return self.s
    
    def A(self, x, k):
        states=[]
        acts = self.acts
        self.acts=None
        for i in range(len(x)):
            temp = super().A(x[i], k)
            states.append(temp)
            
        self.acts=acts
        
        if self.ints:
            states = [states[i][np.array(states[i],dtype=int)==states[i]] for i in range(len(x))]
            
        if self.acts is not None and self.acts_range is not None:
            for i in range(len(x)):
                if self.acts_range[i]!=0.0:
                    below =states[i][states[i]<=self.acts[k][i]][-int((self.acts_range[i]+0.1)*10):]
                    above =states[i][states[i]>self.acts[k][i]][:int(self.acts_range[i]*10)]

                    states[i] = np.append(below,above)
                else:
                    states[i] = self.acts[k][i]
        
        if self.ints and self.furthers is not None:
            states = [states[i][(states[i]+x[i])%self.furthers[i][0]==self.furthers[i][1]] for i in range(len(x))]
            
        actions = np.array(np.meshgrid(*states)).T.reshape(-1,3)
        
        idx = np.round(np.linspace(0, len(actions) - 1, self.max_number_states)).astype(int)

        return actions[idx]
    
    def get_yield(self,k):
        return self.yields.iloc[k].to_numpy()

In [8]:
def tup_sub(tup1,tup2):
    res = [int((tup1[i]-tup2[i])*10)/10 for i in range(len(tup1))] 
    return tuple(res)

def tup_add(tup1,tup2):
    res = [int((tup1[i]+tup2[i])*10)/10 for i in range(len(tup1))] 
    return tuple(res)

def correct_traj_acts(x0,traj,acts):
    traj[0]=x0
    for i in range(len(acts)):
        traj[i+1]=tup_add(traj[i],acts[i])
    
    return traj

def policy_rollout(model, pi, x0):
    cost = 0
    surpluses = []
    J, x, trajectory, actions = 0, x0, [x0], []
    for k in range(model.N):
        u = pi(x, k)
        price = model.g(x, u , True, k)
        surpluses.append([model.get_yield(k)[i]-u[i] for i in range(len(u))])
        J+=price

        x = model.f(x, u, True, k)
        trajectory.append(x) # update the list of the trajectory
        actions.append(u) # update the list of the actions
        
    J += model.gN(x)
    return J, trajectory, actions, np.array(surpluses)

In [9]:
class DP_P2P:
    def __init__(self, start_time, end_time, houses, battery):
        if len(houses)<=1:
            raise Exception("P2P requires more than one house!")
        if False in [house in ["k28", "h16", "h22", "h28", "h32"] for house in houses]:
            raise Exception('All houses should be either "k28", "h16", "h22", "h28", or "h32"')
        if type(battery) is not Battery:
            raise Exception("battery must be a Battery class instance!")
            
        self.start_time = start_time
        self.end_time = end_time
        self.houses = houses
        
        self.battery = battery
        self.N = len(pd.date_range(start=start_time,end=end_time,freq="h"))
        self.results = None
        self.nf = None
        
        merged = merge(houses[0])
        self.sp = merged.loc[Start:End]["SpotPriceDKK"]/1000
        
    def P2P_sol(self, x0, byday=True, verbose=True):
        if self.results is not None:
            return self.results
        
        N=self.N

        all_actions = []
        all_surpluses = np.ones((0,len(self.houses)))

        Start_i = self.start_time
        x0_i = x0
        x0_int_i = tuple([int(x0[i]) for i in range(len(x0))])

        num_loops = int(np.ceil(N/24)) if byday else 1
        remainder = N%24
        length = 24 if byday else N
        for i in range(num_loops):
            if byday and i == num_loops-1:
                length = length if remainder == 0 else remainder

            End_i = pd.date_range(start=Start_i,periods=length,freq="h")[-1]

            if verbose:
                print(f"Period from {Start_i} to {End_i}")
            
            furthers=[[1,0]]
            for t in range(len(self.houses)-1):
                furthers.append([self.battery.max_capacity+1,x0_int_i[t+1]])
            
            for j in range(len(self.houses)):
                if j==0:
                    DPP2P= DP_central(Start_i, End_i, self.houses, deepcopy(self.battery),
                                      degrade=False,ints=True,acts=None,acts_range=None,
                                      furthers=furthers,traj=None,traj_range=None, max_number_states=20)
                    _, pi = DP_stochastic(DPP2P)

                    _, trajectory, actions, _ = policy_rollout(DPP2P, pi=lambda x, k: pi[k][x], x0=x0_int_i)

                    if x0_i != x0_int_i:
                        trajectory = correct_traj_acts(x0_i,trajectory,actions)
                       
                traj_range = [0 if t!=j+1 else -1 for t in range(len(self.houses))]  
                acts_range = [0 if t!=j+1 else self.battery.max_charge for t in range(len(self.houses))]
                
                DPP2P= DP_central(Start_i, End_i, self.houses, deepcopy(self.battery),
                                  degrade=False,ints=False,acts=actions,acts_range=acts_range,
                                  furthers=None,traj=trajectory,traj_range=traj_range, max_number_states=20)
                _, pi = DP_stochastic(DPP2P)

                _, trajectory, actions, surpluses = policy_rollout(DPP2P, pi=lambda x, k: pi[k][x], x0=x0_i)
            
            all_actions = all_actions + actions
            all_surpluses = np.append(all_surpluses,surpluses,axis=0)

            Start_i= pd.date_range(start=End_i,periods=2,freq="h")[-1]
            
            x0_i = trajectory[-1]
            x0_int_i = tuple([int(x0_i[t]) for t in range(len(x0_i))])

        self.results = (all_actions,all_surpluses)
        return self.results
    
    def cost_matrix(self):
        if self.results is None:
            return None
        
        _,surpluses = self.results
        
        nf = pd.DataFrame()
        for i in range(surpluses.shape[1]):
            nf[self.houses[i]] = surpluses[:,i]
            
        new = pd.DataFrame()
        for i in range(len(nf)):
            thing = pd.DataFrame(EnergyMarket(nf.iloc[i].to_dict(),self.sp[i],self.sp[i]+1).get_total_costs(), index=[0])
            new = pd.concat([new,thing],ignore_index=True)
            
        for i in range(len(self.houses)):
            nf['O'+self.houses[i]] = new[self.houses[i]].to_list()
        for i in range(len(self.houses)):
            nf['cumm_O'+self.houses[i]] = nf['O'+self.houses[i]].cumsum()
         
        self.nf = nf
        
        return nf
    
    def total_cost(self):
        if self.nf is None:
            return None
        
        return sum([self.nf['cumm_O'+house][len(self.nf)-1] for house in self.houses])

In [10]:
bat = Battery(max_capacity=13,max_charge=7)

x0 = (0,10,0)

In [11]:
stuff = DP_P2P(Start,End,["h16","h28","k28"],bat)
actions,surpluses = stuff.P2P_sol(x0)

Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
0  

In [12]:
stuff.cost_matrix()

,h16,h28,k28,Oh16,Oh28,Ok28,cumm_Oh16,cumm_Oh28,cumm_Ok28
0,-5.5,2.2,-7.3,13.972145,-3.388858,16.344847,13.972145,-3.388858,16.344847
1,-5.3,-0.3,-6.3,12.791073,0.724023,15.204483,26.763218,-2.664835,31.549330
2,-4.2,-10.3,-5.6,9.371460,22.982391,12.495280,36.134678,20.317555,44.044611
3,-3.8,4.2,-4.1,3.951886,-7.830974,4.263877,40.086564,12.486582,48.308487
4,-3.8,-3.0,-4.1,7.209132,5.691420,7.778274,47.295696,18.178002,56.086762
5,-10.4,-3.1,-4.6,18.984472,5.658833,8.396978,66.280168,23.836835,64.483740
6,3.5,-3.4,-3.4,-5.417072,2.846684,2.846684,60.863096,26.683519,67.330424
7,-10.1,-2.9,-4.3,17.505926,5.026454,7.453018,78.369022,31.709973,74.783442
8,3.7,-3.6,-3.7,-4.673743,2.424420,2.491765,73.695279,34.134393,77.275207
9,-10.2,0.0,-6.0,17.353770,-0.000000,10.208100,91.049049,34.134393,87.483306


In [13]:
stuff.total_cost()

350.25548223317

In [14]:
stuff = DP_P2P(Start,End,["h16","k28","h28"],bat)
actions,surpluses = stuff.P2P_sol(x0)

Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
0  

In [15]:
stuff.cost_matrix()

,h16,k28,h28,Oh16,Ok28,Oh28,cumm_Oh16,cumm_Ok28,cumm_Oh28
0,-5.5,-0.3,-4.8,13.972145,0.762117,12.193872,13.972145,0.762117,12.193872
1,-5.3,-3.3,-4.0,12.791073,7.964253,9.653640,26.763218,8.726370,21.847512
2,-4.2,-5.6,-9.5,9.371460,12.495280,21.197350,36.134678,21.221650,43.044863
3,-3.8,-4.1,4.1,3.951886,4.263877,-7.820574,40.086564,25.485527,35.224288
4,-3.8,-4.1,-3.0,7.209132,7.778274,5.691420,47.295696,33.263801,40.915709
5,-10.4,-4.6,-3.1,18.984472,8.396978,5.658833,66.280168,41.660779,46.574542
6,3.5,-3.4,-3.4,-5.417072,2.846684,2.846684,60.863096,44.507463,49.421226
7,-10.1,-4.3,-2.9,17.505926,7.453018,5.026454,78.369022,51.960481,54.447680
8,3.7,-3.7,-3.6,-4.673743,2.491765,2.424420,73.695279,54.452246,56.872100
9,-10.2,-6.0,0.0,17.353770,10.208100,-0.000000,91.049049,64.660346,56.872100


In [16]:
stuff.total_cost()

349.72784528407

In [17]:
stuff = DP_P2P(Start,End,["h28","h16","k28"],bat)
actions,surpluses = stuff.P2P_sol(x0)

Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
0  

In [18]:
stuff.cost_matrix()

,h28,h16,k28,Oh28,Oh16,Ok28,cumm_Oh28,cumm_Oh16,cumm_Ok28
0,-4.8,1.5,-7.3,12.193872,-2.310585,17.044847,12.193872,-2.310585,17.044847
1,-3.3,-2.3,-6.3,7.964253,5.550843,15.204483,20.158125,3.240258,32.249330
2,-10.3,-5.7,-5.6,22.982391,12.718410,12.495280,43.140516,15.958668,44.744611
3,4.2,-4.1,-4.1,-8.111766,4.263877,4.263877,35.028750,20.222545,49.008487
4,-10.0,-4.8,-7.0,18.971400,9.106272,13.279980,54.000150,29.328817,62.288468
5,4.3,-4.2,-4.2,-6.595186,3.466806,3.466806,47.404964,32.795623,65.755274
6,-9.6,-0.5,-0.2,17.637696,0.918630,0.367452,65.042661,33.714253,66.122726
7,4.6,-4.5,-4.5,-6.276706,3.299670,3.299670,58.765955,37.013923,69.422396
8,-9.6,-7.5,-7.3,16.065120,12.550875,12.216185,74.831075,49.564798,81.638581
9,3.3,-3.2,-3.2,-4.271221,2.244320,2.244320,70.559854,51.809118,83.882901


In [19]:
stuff.total_cost()

344.63385700607

In [20]:
stuff = DP_P2P(Start,End,["h28","k28","h16"],bat)
actions,surpluses = stuff.P2P_sol(x0)

Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
0  

In [21]:
stuff.cost_matrix()

,h28,k28,h16,Oh28,Ok28,Oh16,cumm_Oh28,cumm_Ok28,cumm_Oh16
0,-4.8,-0.3,-5.5,12.193872,0.762117,13.972145,12.193872,0.762117,13.972145
1,-3.3,-3.3,-5.3,7.964253,7.964253,12.791073,20.158125,8.726370,26.763218
2,-10.3,-5.6,-5.7,22.982391,12.495280,12.718410,43.140516,21.221650,39.481629
3,4.2,-4.1,-4.1,-8.111766,4.263877,4.263877,35.028750,25.485527,43.745505
4,-10.0,-4.1,-4.8,18.971400,7.778274,9.106272,54.000150,33.263801,52.851777
5,4.3,-4.2,-4.2,-6.595186,3.466806,3.466806,47.404964,36.730607,56.318583
6,-9.6,-4.1,-0.5,17.637696,7.532766,0.918630,65.042661,44.263373,57.237213
7,4.6,-4.5,-4.5,-6.276706,3.299670,3.299670,58.765955,47.563043,60.536884
8,-9.6,-9.8,-4.6,16.065120,16.399810,7.697870,74.831075,63.962854,68.234754
9,3.3,-3.2,-3.2,-4.271221,2.244320,2.244320,70.559854,66.207173,70.479073


In [22]:
stuff.total_cost()

340.80319536855995

In [23]:
stuff = DP_P2P(Start,End,["k28","h28","h16"],bat)
actions,surpluses = stuff.P2P_sol(x0)

Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
0  

In [24]:
stuff.cost_matrix()

,k28,h28,h16,Ok28,Oh28,Oh16,cumm_Ok28,cumm_Oh28,cumm_Oh16
0,-7.3,2.2,-5.5,16.344847,-3.388858,13.972145,16.344847,-3.388858,13.972145
1,-6.3,-0.3,-7.1,15.204483,0.724023,17.135211,31.549330,-2.664835,31.107356
2,-5.6,-10.3,-2.9,12.495280,22.982391,6.470770,44.044611,20.317555,37.578126
3,-4.1,4.2,-4.1,4.263877,-8.111766,4.263877,48.308487,12.205790,41.842003
4,-4.1,-3.0,-3.0,7.778274,5.691420,5.691420,56.086762,17.897210,47.533423
5,-10.5,-2.7,-4.5,19.167015,4.928661,8.214435,75.253776,22.825871,55.747858
6,3.8,-3.7,-3.7,-5.894310,3.097862,3.097862,69.359466,25.923733,58.845720
7,-11.3,-1.3,-2.7,19.585838,2.253238,4.679802,88.945304,28.176971,63.525522
8,4.2,-4.1,-4.1,-5.252910,2.761145,2.761145,83.692394,30.938116,66.286667
9,-8.4,0.8,-5.6,13.491340,-0.561080,9.527560,97.183734,30.377036,75.814227


In [25]:
stuff.total_cost()

344.11503813937

In [26]:
stuff = DP_P2P(Start,End,["k28","h16","h28"],bat)
actions,surpluses = stuff.P2P_sol(x0)

Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
0  

In [27]:
stuff.cost_matrix()

,k28,h16,h28,Ok28,Oh16,Oh28,cumm_Ok28,cumm_Oh16,cumm_Oh28
0,-7.3,1.5,-4.8,17.044847,-2.310585,12.193872,17.044847,-2.310585,12.193872
1,-6.3,-2.3,-4.0,15.204483,5.550843,9.653640,32.249330,3.240258,21.847512
2,-5.6,-4.2,-9.5,12.495280,9.371460,21.197350,44.744611,12.611718,43.044863
3,-4.1,-3.8,4.1,4.263877,3.951886,-7.820574,49.008487,16.563604,35.224288
4,-4.1,-3.8,-3.0,7.778274,7.209132,5.691420,56.786762,23.772736,40.915709
5,-10.5,-4.5,-2.7,19.167015,8.214435,4.928661,75.953776,31.987171,45.844370
6,3.8,-3.7,-3.7,-5.894310,3.097862,3.097862,70.059466,35.085033,48.942232
7,-11.3,-2.7,-1.3,19.585838,4.679802,2.253238,89.645304,39.764835,51.195470
8,4.2,-4.1,-4.1,-5.252910,2.761145,2.761145,84.392394,42.525980,53.956615
9,-8.4,-5.6,0.8,13.491340,9.527560,-0.561080,97.883734,52.053540,53.395535


In [28]:
stuff.total_cost()

341.64434296643003